## DATA PRE-PROCESSING

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import glob
import cv2
import os
import matplotlib.pyplot as plt

In [ ]:

###############################################################

def read_image_data(file_path, bool_images):
    images_arr = []
    id_genres_list_ = []
    '''lists all the directories in file_path'''
    inner_dirs = [path for path in glob.glob(file_path)]
    
    if bool_images:
        for path in inner_dirs:

            images = []
            images_names_list = (os.listdir(path))
            images_arr = read_images(path, images_names_list, images_arr)

        return (images_arr)
    
    else:
        for path in inner_dirs:
            
            id_genres_list_ = id_genres_list_ + func_to_read_id_genre_data(path)
            
        
        return id_genres_list_

    

    
###############################################################

def read_images(path, images_names, images_arr):
    for image_name in images_names:
        resized_image = cv2.resize(cv2.imread(path + "/" + image_name), (300, 400))
        resized_image = resized_image / 255
        img_name_ = image_name[0 : image_name.index('.jpg')]
        
        '''adding the image and its id as an ordered pair to the images_arr'''
        images_arr.append((img_name_, resized_image))

    return images_arr




###############################################################

def read_image_genre_id(path, encoding_):
    #genres_set = {}
    id_genres_list = []
    id_val = ''
    genres = []
    id_flag = False
    genre_flag = False
  
    with open(path, encoding = encoding_) as file:
        for line in file:
            
            arr = line.split(':')
            if len(arr) > 1:
                '''removes non-alphanumerics from the string'''
                id_ = ''.join(e for e in arr[0] if e.isalnum())
                if (id_) == 'imdbID':
                    id_flag = True
                    id_val = ''.join(e for e in arr[1] if e.isalnum())
                    
                if id_ == 'Genre':
                    genre_flag = True
                    genres_ = arr[1][0 : -2].split(',')
                    for genre in genres_:
                        word = ''.join(e for e in genre if e.isalnum())
                        genres.append(word)

                if id_flag and genre_flag:

                    id_genre_pair = (id_val, genres)
                    id_genres_list.append(id_genre_pair)
                    
                    
                    id_flag = False
                    genre_flag = False
                    id_val = ''
                    genres = []

        
        return id_genres_list

    


###############################################################

def func_to_read_id_genre_data(path):
    
    try:
        return read_image_genre_id(path, 'utf-16-le')
    
    except UnicodeDecodeError:
        return read_image_genre_id(path, 'utf-8')
    


    
    
###############################################################

def convert_id_genre_list_to_df(imageid_genre_list):
    id_genre_dicts_list = []
    for id_genre in image_id_genre_list:
        id_genre_dict = {}
        id_genre_dict['movie_Id'] = id_genre[0]
        for genre in id_genre[1]:
            id_genre_dict[genre] = 1
        
        id_genre_dicts_list.append(id_genre_dict)
    
    return pd.DataFrame(id_genre_dicts_list)



###############################################################

def convert_image_list_to_df(image_list):
    return pd.DataFrame(image_list, columns = ['movie_Id', 'pixel_data'])



###############################################################

def fill_NaNs_with_Zeros(id_genre_df):
    return id_genre_df.fillna(0)


###############################################################

def merge_image_data_id_genre_Dfs(image_df, id_genre_df):
    return pd.merge(image_df, id_genre_df, on = 'movie_Id')




#------------------*********************************--------------------------####


def get_features_and_labels(image_id_genre_data_df):
    feature_columns_ = []
    df_columns = image_id_genre_data_df.columns
    label_columns = [column for column in df_columns if column not in ['movie_Id', 'pixel_data']]
    
    pixel_vals = image_id_genre_data_df['pixel_data'].values
    for val in pixel_vals:
        feature_columns_.append(val)
        
    #feature_data = image_id_genre_data_df.loc[:, 'pixel_data']
    label_data   = image_id_genre_data_df.loc[:, label_columns]
    
    return np.array(feature_columns_), label_data


In [ ]:
image_list = read_image_data("/Users/vijay/Downloads/Datasets/movie_poster_data/*", True)
image_id_genre_list = read_image_data("/Users/vijay/Downloads/Datasets/ground_truth/*", False)

id_genre_df = fill_NaNs_with_Zeros(convert_id_genre_list_to_df(image_id_genre_list))
image_df = convert_image_list_to_df(image_list)

del(image_list)
del(image_id_genre_list)

image_id_genre_pixel_data_df = merge_image_data_id_genre_Dfs(image_df, id_genre_df)

del(image_df)
#del(image_id_genre_list)

X_data, label_data = get_features_and_labels(image_id_genre_pixel_data_df)

del(image_id_genre_pixel_data_df)

In [ ]:
import math
from sklearn.model_selection import train_test_split

def split_the_train_data(train_X, train_Y):
    train_x, train_y, test_x, test_y = train_test_split(train_X, train_Y, test_size = 0.2, 
                                                        random_state = 42)
    return [[train_x, test_x], [train_y, test_y]]



def get_train_and_label_data(train_data, test_data):

    no_of_rows = np.shape(train_data)[0]
#     no_of_train_samples = int(math.modf(no_of_rows * 0.8)[1])
    no_of_train_samples = 600
    train_X = train_data[0 : no_of_train_samples]
    train_Y = test_data[0 : no_of_train_samples]
    
    #test_X = train_data.loc[35000: ,]
    test_X = train_data[no_of_train_samples : ]
    test_Y = test_data[no_of_train_samples : ]
    
    return [[train_X, train_Y], [test_X, test_Y]]



train_test_data  = get_train_and_label_data(X_data, label_data.values)
train_X, train_Y = train_test_data[0]
test_X, test_Y   = train_test_data[1]

    

## TENSORFLOW MODEL

In [ ]:

x_data_shape = np.shape(X_data)
batch_size = 100
height = x_data_shape[1]
width  = x_data_shape[2]
no_of_channels = x_data_shape[3]
no_of_labels = np.shape(label_data)[1]

x = tf.placeholder(tf.float32, [None, 400 * 300 * 3])
x_rs = tf.reshape(x, [-1, 400, 300, 3])

y = tf.placeholder(tf.float32, [None, no_of_labels])
y_rs = tf.reshape(y, [-1, no_of_labels])

keep_prob = tf.placeholder(tf.float32)



In [ ]:


def get_batch_size(no_of_rows):
    batch_size = 1
    for i in range(2, 51):
        if no_of_rows % i ==0:
            if i > batch_size:
                batch_size = i
            
    return batch_size



def get_total_no_of_neurons(shape_of_conv_out):
    neurons = 1
    for num in shape_of_conv_out:
        if num is not None:
            neurons = neurons * num
    return neurons



def get_weights_and_bias_for_conv_and_pooling():
    w_1 = tf.Variable(tf.truncated_normal([5, 5, no_of_channels, 16], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'w_1')
    b_1 = tf.Variable(tf.constant(0.1, shape = [16], dtype = tf.float32), name = 'b_1')
    
    w_2 = tf.Variable(tf.truncated_normal([5, 5, 16, 32], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'w_2')
    b_2 = tf.Variable(tf.constant(0.1, shape = [32], dtype = tf.float32), name = 'b_2')
    
    w_3 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'w_3')
    b_3 = tf.Variable(tf.constant(0.1, shape = [64], dtype = tf.float32), name = 'b_3')
    
    w_4 = tf.Variable(tf.truncated_normal([5, 5, 64, 64], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'w_3')
    b_4 = tf.Variable(tf.constant(0.1, shape = [64], dtype = tf.float32), name = 'b_3')
    
    return [[w_1, b_1], [w_2, b_2], [w_3, b_3], [w_4, b_4]]



def get_weights_and_bias_for_fc_layers(conv_result_shape):
    
    total_no_of_neurons = get_total_no_of_neurons(conv_result_shape)
    
    fc_w_1 = tf.Variable(tf.truncated_normal([total_no_of_neurons, 128], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'fc_w_1')
    fc_b_1 = tf.Variable(tf.constant(0.1, shape = [128], dtype = tf.float32), name = 'fc_b_1')
    
    fc_w_2 = tf.Variable(tf.truncated_normal([128, 64], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'fc_w_2')
    fc_b_2 = tf.Variable(tf.constant(0.1, shape = [64], dtype = tf.float32), name = 'fc_b_1')
    
    fc_w_3 = tf.Variable(tf.truncated_normal([64, no_of_labels], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'fc_w_3')
    fc_b_3 = tf.Variable(tf.constant(0.1, shape = [no_of_labels], dtype = tf.float32), name = 'fc_b_3')
    
    return [[fc_w_1, fc_b_1], [fc_w_2, fc_b_2], [fc_w_3, fc_b_3]]





def apply_convolution_on_the_data(x, w, b):
    
    convoluted_image = tf.nn.conv2d(x, w, strides = [1, 1, 1, 1], padding = 'SAME')
    CI_with_bias     = tf.nn.bias_add(convoluted_image, b)
    activated_CI     = tf.nn.relu(CI_with_bias)
    return activated_CI



def apply_max_pooling_on_convolution(convolution, k = 2):
    return tf.nn.max_pool(convolution, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'SAME')




def perform_convolution_and_pooling_on_data():
    
    weights_biases = get_weights_and_bias_for_conv_and_pooling()
    w_1, b_1       = weights_biases[0]
    w_2, b_2       = weights_biases[1]
    w_3, b_3       = weights_biases[2]
    w_4, b_4       = weights_biases[3]
    
    
    conv_1 = apply_convolution_on_the_data(x_rs, w_1, b_1)
    pool_1 = apply_max_pooling_on_convolution(conv_1)
    norm_1 = tf.nn.lrn(pool_1, depth_radius = 4, bias = 1.0, alpha = 0.002 / 9.0, beta = 0.75)
    
    
    conv_2 = apply_convolution_on_the_data(norm_1, w_2, b_2)
    pool_2 = apply_max_pooling_on_convolution(conv_2)
    norm_2 = tf.nn.lrn(pool_2, depth_radius = 4, bias = 1.0, alpha = 0.002 / 9.0, beta = 0.75)
    
    conv_3 = apply_convolution_on_the_data(norm_2, w_3, b_3)
    pool_3 = apply_max_pooling_on_convolution(conv_3)
    norm_3 = tf.nn.lrn(pool_3, depth_radius = 4, bias = 1.0, alpha = 0.002 / 9.0, beta = 0.75)
    
    conv_4 = apply_convolution_on_the_data(norm_3, w_4, b_4)
    pool_4 = apply_max_pooling_on_convolution(conv_3)
    norm_4 = tf.nn.lrn(pool_4, depth_radius = 4, bias = 1.0, alpha = 0.002 / 9.0, beta = 0.75)

    return norm_4


def form_fc_layers_and_perfom_activations(conv_result):
    
    shape_of_conv_result   = conv_result.get_shape().as_list()
    fc_weights_biases = get_weights_and_bias_for_fc_layers(shape_of_conv_result)
    
    fc_w_1, fc_b_1    = fc_weights_biases[0]
    fc_w_2, fc_b_2    = fc_weights_biases[1]
    fc_w_3, fc_b_3    = fc_weights_biases[2]
    
    
    conv_res_reshaped = tf.reshape(conv_result, [-1, fc_w_1.get_shape().as_list()[0]])
    fc_layer_1        = tf.add(tf.matmul(conv_res_reshaped, fc_w_1), fc_b_1)
    fc_relu_1         = tf.nn.relu(fc_layer_1)

    
    fc_layer_2        = tf.add(tf.matmul(fc_relu_1, fc_w_2), fc_b_2)
    fc_relu_2         = tf.nn.relu(fc_layer_2)
    fc_2_dropout      = tf.nn.dropout(fc_relu_2, keep_prob)
    
    fc_layer_out      = tf.add(tf.matmul(fc_2_dropout, fc_w_3), fc_b_3)
    
    return fc_layer_out


def CNN_movie_image():
    
    conv_result   = perform_convolution_and_pooling_on_data()
    fc_layer_out  = form_fc_layers_and_perfom_activations(conv_result)
    
    return fc_layer_out


def perform_optimization():
    fc_layer_res         = CNN_movie_image()
    cost                 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y_rs, logits = fc_layer_res))
    optimizer            = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
    correctly_predicted  = tf.equal(tf.argmax(fc_layer_res, 1), tf.argmax(y_rs, 1))
    accuracy             = tf.reduce_mean(tf.cast(correctly_predicted, tf.float32))
    
    return fc_layer_res, optimizer, accuracy




def run_CNN_for_movie_Genre():
    with tf.Session() as session:
        fc_layer_result, optimizer, accuracy  = perform_optimization()
        session.run(tf.global_variables_initializer())
        saver      = tf.train.Saver(tf.trainable_variables())
        #batch_size = get_batch_size(np.shape(train_X)[0])
        #print('batch size ' + str(batch_size))
        for epoch in range(0, 3):
            for i in range(0, len(train_X), batch_size):
                
                train_x         = train_X[i : i + batch_size]
                train_y         = train_Y[i : i + batch_size]
                
                _, accuracy_val = session.run([optimizer, accuracy], feed_dict = {x_rs : train_x, 
                                                                                  y_rs : train_y, 
                                                                                  keep_prob: 0.5})

            if epoch % 1 == 0:
                print(epoch, accuracy_val)

    
        save_path = saver.save(session, "./mov_gen_img_model")
        
        return fc_layer_result
        


In [ ]:
fc_layer_out = run_CNN_for_movie_Genre()

In [ ]:
with tf.Session() as session:
    meta_graph = tf.train.import_meta_graph('./mov_gen_img_model.meta')
    meta_graph.restore(session, tf.train.latest_checkpoint('./'))
    
    prediction = session.run(fc_layer_out, feed_dict = {x_rs : test_X})
    print((session.run(tf.nn.sigmoid(prediction))))